# cuPauliProp 예제 - Pauli Propagation 완전 가이드

## 개요
**목표**: Pauli propagation을 사용한 기댓값 계산 완전 이해

**핵심 이론**: Heisenberg picture에서 $\langle 0|U^\dagger O U|0\rangle$ 계산

**학습 순서**:
1. Setup: 라이브러리 및 기본 파라미터
2. Observable 초기화: Pauli string 인코딩
3. Buffer 준비: GPU 메모리 할당
4. Circuit 구성: Gate operators
5. Propagation: Observable 변환
6. 계산: Trace 계산
7. 검증: 상태 벡터 비교
8. 정리: 리소스 해제

---

## 핵심 개념

**Pauli 인코딩**:
- X mask: X 또는 Y가 있는 큐비트 위치
- Z mask: Z 또는 Y가 있는 큐비트 위치
- 예: Z₀ = (X=0b00, Z=0b01), Y₀ = (X=0b01, Z=0b01)

**Trace 계산**:
- ⟨0|I|0⟩ = 1 (항등원소)
- ⟨0|X|0⟩ = 0 (직교)
- ⟨0|Y|0⟩ = 0 (직교)
- ⟨0|Z|0⟩ = 1 (고유상태)

**Gate 종류**:
- Pauli Rotation: RX(θ), RY(θ), RZ(θ)
- Clifford: H, X, Y, Z, S, S†, CNOT 등

## 초기화 및 정리 함수

**사용 방법:**

1. **노트북 시작 및 종료시**: 다음 셀의 `cleanup_cupauli()`를 호출하여 리소스를 정리
   ```python
   cleanup_cupauli()
   ```
   이 함수는:
   - 기존 게이트, 확장, 워크스페이스, 핸들을 정리합니다
   - 새로운 handle을 생성하고 반환합니다

2. **시작 단계에서 리소스 정리 후 handle 생성시**: `make_handle` 함수를 호출
   ```python
   make_handle()
   ```
   이 함수는 작업에 필요한 handle을 만듭니다.

## Observable 초기화
### 초기 Observable 설정 

**Pauli 연산자 인코딩**:
- Pauli string은 X mask와 Z mask로 표현
- X mask: X 또는 Y 연산자가 있는 큐비트 위치
- Z mask: Z 또는 Y 연산자가 있는 큐비트 위치

**예시 (단일 항)**:
- Z₀: X_mask=0b00, Z_mask=0b01 → `[0b00, 0b01]`
- Z₁: X_mask=0b00, Z_mask=0b10 → `[0b00, 0b10]`
- X₀: X_mask=0b01, Z_mask=0b00 → `[0b01, 0b00]`
- Y₀: X_mask=0b01, Z_mask=0b01 → `[0b01, 0b01]`

**여러 항의 선형 합**:

API 문서에 따르면, 여러 Pauli 항은 다음 순서로 저장:
```python
# ZZZ + XYI 예시 (API 문서):
# 항1(ZZZ): X_mask=000, Z_mask=111
# 항2(XYI): X_mask=110, Z_mask=010
# → [000, 111, 110, 010]
# 즉, [항1_X, 항1_Z, 항2_X, 항2_Z, ...]
```

**데이터 구조 (정확한 형식)**:
```python
# 단일 항: [X_mask, Z_mask]
d_input_pauli = cp.array([0b00, 0b01], dtype=cp.uint64)  # Z₀

# 여러 항 (예: Z₀ + X₁):
d_input_pauli = cp.array([
    0b00, 0b01,  # 항1: Z₀ [X_mask, Z_mask]
    0b10, 0b00   # 항2: X₁ [X_mask, Z_mask]
], dtype=cp.uint64)
d_input_coef = cp.array([1.0, 1.0], dtype=cp.float64)
```

In [ ]:
# Pauli 데이터 구조: [X_mask, Z_mask] 순서

import cupy as cp
print("✓ Observable 설정: Z₅")
d_input_pauli = cp.array([0b00, 0b10000], dtype=cp.uint64) # Z_5
d_input_coef = cp.array([1.0], dtype=cp.float64) 

print("✓ Observable 설정: Z₅")

# 다른 예시들 (주석):
d_input_pauli = cp.array([0b10, 0b01], dtype=cp.uint64)  # Z₀X₁

# 여러 항의 선형 합 예시:
d_input_pauli = cp.array([
    0b01, 0b00,  # Z₀: [X_mask, Z_mask]
    0b00, 0b10   # X₁: [X_mask, Z_mask]
], dtype=cp.uint64)
d_input_coef = cp.array([1.0, 0.3], dtype=cp.float64)  # 1.0×Z₀ + 0.3×X₁


✓ Observable 설정: Z₅
  X mask: 0b0
  Z mask: 0b10000
  계수: 1.0


## 💡 Workspace 이해하기

**Workspace가 필요한 이유**:
1. **중간 계산 결과 저장**: Gate 연산 시 임시 데이터 저장
2. **Term 분기 처리**: Rotation gate가 1개 term을 2개로 분기할 때 임시 공간 필요
3. **정렬/병합**: 중복 term 제거 시 임시 버퍼

**Layer 증가 시 문제**:
- Layer 2: 24 rotations → 최악의 경우 2^24 = 16M terms (실제는 훨씬 적음)
- Layer 3: 36 rotations → 최악의 경우 2^36 = 68B terms (불가능!)
- **해결책**: 중간에 sort=True로 중복 제거 필요

**최적화 전략**:
1. **주기적 정리**: 매 layer 후 중복 term 병합
2. **Threshold 설정**: 작은 계수 제거
3. **Max terms 증가**: GPU 메모리 허용 범위 내에서

## Buffer 및 Expansion 준비
### Pauli Expansion 생성

Pauli expansion은 GPU에서 Pauli 연산자를 처리하기 위한 자료구조입니다.

**중요**: Non-Clifford gate (예: RY, RZ)는 Pauli term을 분기시켜 2배씩 증가시킵니다.
- 초기: Z₀ (1개 term)
- RY 적용 후: 여러 Pauli term으로 확장 가능
- 충분한 `max_terms`를 설정해야 합니다

## Circuit 구성
### 양자 회로 구성

**회로**: H(0) → CNOT(0,1) → RY(π/4, qubit 1)

#### Gate 종류

##### 1. Pauli Rotation Gate
- 형태: $e^{-i\theta P/2}$, P는 Pauli 연산자
- API: `create_pauli_rotation_gate_operator`
- Pauli 종류: 0=I, 1=X, 2=Z, 3=Y

##### 2. Clifford Gate
- CNOT, CZ, H, S 등
- API: `create_clifford_gate_operator`
- Gate 종류:
  - 0: I (CNOT)
  - 1: X
  - 2: Z
  - 3: Y
  - 4: H
  - 5: S
  - 7: CX [target, control] 순임;;;
  - 8: CZ
  - 9: CY
  - 10: SWAP
  - 11: ISWAP
  - 12: SQRTX
  - 13: SQRTZ
  - 14: SQRTY

In [ ]:
gate_ops = []

### 포인터 사용 방식 ###
qubit_0 = np.array([0], dtype=np.int32)
pauli_y = np.array([3], dtype=np.int32)  # CUPAULIPROP_PAULI_Z (RZ 사용)
ry = cupp.create_pauli_rotation_gate_operator(
    handle, theta, 1, qubit_0.ctypes.data, pauli_y.ctypes.data)


### 파울리 회전 게이트 ###
rot = cupp.create_pauli_rotation_gate_operator(handle, degree, 1, [qubit], [axis])
gate_ops.append(rot)

### 클리포드 게이트 ###
h = cupp.create_clifford_gate_operator(handle, 4, [0]) # H on qubit 0# 
gate_ops.append(h)# for i in range(1, 2):

# CNOT (순서: [control, target])
cnot = cupp.create_clifford_gate_operator(handle, 7, [target, control]) 
gate_ops.append(cnot)

## Pauli Propagation 및 계산
### Pauli Propagation (Heisenberg Picture)

### 이론

**목표**: $\langle\psi|Z_0|\psi\rangle$ 계산, 여기서 $|\psi\rangle = U|0\rangle$

**Heisenberg picture**:
$$
\langle\psi|Z_0|\psi\rangle = \langle 0|U^\dagger Z_0 U|0\rangle
$$

즉, 상태를 변환하는 대신 Observable을 변환합니다.

### 구현

**중요**: Gate를 역순으로 adjoint 적용
- 회로: $G_1 G_2 G_3$ → U = $G_3 G_2 G_1$
- Propagation: $Z_0$ → $G_3^\dagger Z_0 G_3$ → $G_2^\dagger (G_3^\dagger Z_0 G_3) G_2$ → ...

**API 설정**:
- `adjoint=1`: $G^\dagger O G$ 계산
- 역순 루프: `range(len(circuit)-1, -1, -1)`